In [1]:
## Primeiro criando o Lexer
def cmd(command):
    from os import popen
    print(popen(command).read())
from os import listdir as ls
"""
To make jflex available as an app. Unzip jflex from http://jflex.de/download.html 
and add JFLEX_HOME to your and %JFLEX_HOME%\bin to your path

SET JFLEX_HOME=c:\jflex\
"""    
cmd('echo %JFLEX_HOME%')
cmd('jflex --version')
cmd('jflex cup/lcalc.flex')
ls('cup')

jflex

This is JFlex 1.6.1

Reading "cup\lcalc.flex"
Constructing NFA : 46 states in NFA
Converting NFA to DFA : 
.................
19 states before minimization, 14 states in minimized DFA
Writing code to "cup\Lexer.java"



['delfiles.bat',
 'lcalc.flex',
 'Lexer.java',
 'Main.java',
 'test.txt',
 'ycalc.cup']

#### Definição CUP - Regras Gramática

Considere que a gramática e ações semânticas serão feitas no **ycalc.cup**.  
Considere o seguinte código: 
```cup
   #Gramática exemplo (calculadora) 
    expr_list ::=   expr_list expr_part
                 | expr_part
    expr_part ::=   expr SEMI
    expr      ::=   expr PLUS factor
                 | expr MINUS factor
                 | factor
    factor    ::=   factor TIMES term
                 | factor DIVIDE term
                 | term
    term     ::=    LPAREN expr RPAREN
                 | NUMBER
                 | ID     
                 
    .... 
    
    expr_list ::= expr_list expr_part
                 |
                 expr_part;
    
    expr_part ::= expr:e
                 {: System.out.println(" = " + e); :}
                 SEMI
                 ;
    
    expr      ::= expr:e PLUS factor:f
                 {: RESULT = new Integer(e.intValue() + f.intValue()); :}
                 |
                 expr:e MINUS factor:f
                 {: RESULT = new Integer(e.intValue() - f.intValue()); :}
                 |
                 factor:f
                 {: RESULT = new Integer(f.intValue()); :}
                 ;
    
    factor    ::= factor:f TIMES term:t
                 {: RESULT = new Integer(f.intValue() * t.intValue()); :}
                 |
                 factor:f DIVIDE term:t
                 {: RESULT = new Integer(f.intValue() / t.intValue()); :}
                 |
                 term:t
                 {: RESULT = new Integer(t.intValue()); :}
                 ;
    term      ::= LPAREN expr:e RPAREN
                 {: RESULT = e; :}
                 |
                 NUMBER:n
                 {: RESULT = n; :}
                 |
                 ID:i
                 {: RESULT = i; :}
                 ;                 
    ....
```

No exemplo em questão... as regras estão misturadas... Não entendo se isso é algo bom ou ruim.
No arquivo .cup ele imprime apenas '= e' (mas calcula todos os passos inclusive +, /, *, -);
No arquivo .lex ele imprime os tokens ao encontra-los. Mas apenas os imprime e os "Retorna" usando funções do cup. 
A montagem do código intermediário deve ser feita portanto no código contido em *.cup
    

##### Regras léxicas - JFLEX + code
```    
    No arquivo .lex ele imprime '+' ou um numero que faz parte de uma expressão '(4) + 3 = 7'
        "+"                { System.out.print(" + "); return symbol(sym.PLUS); }
        {dec_int_lit}      { System.out.print(yytext());
                         return symbol(sym.NUMBER, new Integer(yytext())); }

    ;
```

Ou seja, o arquivo .lex irá encontrar os tokens e lexemas do arquivo fonte e irá atribuir valores a eles. Gerando uma arvore de derivação anotada. 

No arquivo .cup regras/ações semanticas irão sintetizar o código em assembly. 
Gerando a saida necessária. 



In [2]:
f = open('README.md')
print(f.read())
f.close()

A small JFlex+Cup example

It comes from a short article series in the Linux Gazette by Richard
A. Sevenich and Christopher Lopes, titled "Compiler Construction
Tools". The article series starts at
Small changes and updates to newest JFlex+Cup versions by Gerwin Klein





Files:

cup/Main.java       demo of a main program
cup/lcalc.flex      the lexer spec
cup/output.good     how the output should look like for the test
cup/ycalc.cup       the parser spec
cup/test.txt        sample input for testing


> To Compile
		~> Added JFLEX to PATH optional 
		jflex lcalc.flex 										(Generates Lexer.java )                                                
		java -cp <path_to_cup.jar>;. java_cup.Main < ycalc.cup 	(Receives the language grammar specs )
		javac -cp <path_to_cup.jar>;. Main.java 				(Compiles Main class to would be equivalent to a compiler/interpreter )

> To run: 
		java -cp <path_to_cup.jar>;. Main test.txt              (runs test.txt and outputs result)


In [3]:
#cmd('java -cp ../lib;. java_cup.Main < cup/ycalc.cup')
#cmd('java -cp %{0}%\lib;. java_cup.Main < cup/ycalc.cup')

In [4]:
from os import chdir, getcwd, sep, system

def deleteFiles():
    # Deletes - 'parser.java', 'sym.java', 'Lexer.java', 'Lexer.java~', '*.class'
    import subprocess
    subprocess.check_output('delfiles.bat')
        
retdir = getcwd()
chdir('cup')
deleteFiles()
filesBefore = ls()
cmd('jflex lcalc.flex')
cmd('java -cp %JFLEX_HOME%\lib\java-cup-11a.jar;. java_cup.Main < ycalc.cup')
filesAfter = ls()
assert len(filesBefore) <= len(filesAfter)
print("New files = {0}".format(set(filesAfter).difference(set(filesBefore))))

chdir(retdir) # goes back to correct dir
print("Current directory : " + retdir)


Reading "lcalc.flex"
Constructing NFA : 46 states in NFA
Converting NFA to DFA : 
.................
19 states before minimization, 14 states in minimized DFA
Writing code to "Lexer.java"


New files = {'Lexer.java', 'parser.java', 'sym.java'}
Current directory : C:\Users\Paulo\academic\GOCompiler\SimplestPossibleExample


Agora que temos o ``` sym.java, parser.java e Lexer.java ``` 

  After the scanner, lcalc.flex, and the parser, ycalc.cup, have been created. 
  > javac Main.java // Isso compilara o arquivo principal de tradução de código #calc - result 
  
  > java Main test.txt

  
  

In [5]:
chdir('cup') #resumindo
cmd('jflex lcalc.flex') # generates Lexer 
cmd('java -cp %JFLEX_HOME%\lib\java-cup-11a.jar;. java_cup.Main < ycalc.cup') # generates parser.java, sym.java
cmd('javac -cp %JFLEX_HOME%\lib\java-cup-11a.jar;. Main.java') # generates Translator that Creates a parser in the form | new parser(new Lexer(new FileReader(argv[0])));
cmd('java -cp %JFLEX_HOME%\lib\java-cup-11a.jar;. Main test.txt') # passes test.txt to the Lexer > parser 

dirty = set(ls())
deleteFiles()
clean = set(ls())
print("Deleting files: " + str(dirty - clean))
chdir(retdir)

Reading "lcalc.flex"
Constructing NFA : 46 states in NFA
Converting NFA to DFA : 
.................
19 states before minimization, 14 states in minimized DFA
Old file "Lexer.java" saved as "Lexer.java~"
Writing code to "Lexer.java"



2 + 4 = 6
5 *  ( 6 - 3 )  + 1 = 16
6 / 3 * 5 + 20 = 30
4 * 76 / 31 = 9
1 - 1 - 1 = -1

Deleting files: {'CUP$parser$actions.class', 'parser.java', 'sym.java', 'Lexer.java~', 'sym.class', 'Lexer.java', 'Main.class', 'parser.class', 'Lexer.class'}


##### RESUMO 

1. [Adicionar regras léxicas](#Regras-léxicas---JFLEX-+-code) e adicionar ações para "anotar" a arvore de derivação do programa fonte. 
2. [Adicionar regras no arquivo .cup](#Definição-CUP---Regras-Gramática) que vai pegar os valores anotados dos nós e utilizalos para gerar código fonte assembly. 